In [4]:
# import necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import  train_test_split
from sklearn.metrics import classification_report
# from pyimagesearch import simplepreprocessor
# from pyimagesearch import simpledatasetloader
from imutils import paths
import os
import glob
import cv2
import numpy as np
from src.features.build_features import getLBPimage

def getListOfFiles(dirName):
    # create a list of file and sub directories
    # names in the given directory
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)

    return allFiles

def create_training_data():
    imagePaths = getListOfFiles("../data/raw") ## Folder structure: datasets --> sub-folders with labels name
    #print(imagePaths)

    data = []
    lables = []
    c = 0 ## to see the progress
    for image in imagePaths:

        lable = os.path.split(os.path.split(image)[0])[1]
        lables.append(lable)

        img = cv2.imread(image)
    #     print('image is ', image)

        img = getLBPimage(img)
        img = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)        
#         vecimgLBP = imgLBP.flatten()
        data.append(img[0])
        c=c+1
        print(c)
#         if c > 3: 
#             break
    return data, lables


In [39]:
data, lables = create_training_data()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [40]:
# print(lables)

# encode the labels as integer
data = np.array(data)
lables = np.array(lables)

le = LabelEncoder()
lables = le.fit_transform(lables)

myset = set(lables)
print(myset)

dataset_size = data.shape[0]
data = data.reshape(dataset_size,-1)

print(data.shape)
print(lables.shape)
print(dataset_size)

(trainX, testX, trainY, testY ) = train_test_split(data, lables, test_size= 0.25, random_state=42)

model = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
model.fit(trainX, trainY)

print(classification_report(testY, model.predict(testX), target_names=le.classes_))

{0, 1}
(29, 100)
(29,)
29
              precision    recall  f1-score   support

          dn       0.67      0.80      0.73         5
          pc       0.50      0.33      0.40         3

    accuracy                           0.62         8
   macro avg       0.58      0.57      0.56         8
weighted avg       0.60      0.62      0.60         8



In [47]:
score = model.score(testX, testY)
score

0.625

In [48]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, trainX, trainY, cv=10)

/Users/bruceliu/projects/milk_cls/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


array([0.66666667, 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.        ])

In [50]:
clf.fit(trainX, trainY)

DecisionTreeClassifier(random_state=0)

In [51]:
score = clf.score(testX, testY)
score

0.375

In [53]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(trainX, trainY)
clf.score(testX, testY)

0.5

In [6]:
from src.features.build_features import circle_to_lbp
from PIL import Image, ImageDraw

def extract_infor_circle(img_origin, x, y, r):
    img_arr = img_origin
    h,w = img_arr.shape[1], img_arr.shape[0]
    lum_img = Image.new('L',[h,w] ,0) 
    draw = ImageDraw.Draw(lum_img)

    leftUpPoint = (x-r, y-r)
    rightDownPoint = (x+r, y+r)
    twoPointList = [leftUpPoint, rightDownPoint]
    draw.ellipse(twoPointList, fill=255)
    lum_img_arr = np.array(lum_img)
    final_img_arr = np.dstack((img_arr, lum_img_arr))
    return final_img_arr

def getLBPimage(image):
    ### Step 0: Step 0: Convert an image to grayscale
    gray_image = image
#     cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imgLBP = np.zeros_like(gray_image)
    neighboor = 3 
    for ih in range(0,image.shape[0] - neighboor):
        for iw in range(0,image.shape[1] - neighboor):
            ### Step 1: 3 by 3 pixel
            img          = gray_image[ih:ih+neighboor,iw:iw+neighboor]
            center       = img[1,1]
            img01        = (img >= center)*1.0
            img01_vector = img01.T.flatten()
            # it is ok to order counterclock manner
            # img01_vector = img01.flatten()
            ### Step 2: **Binary operation**:
            img01_vector = np.delete(img01_vector,4)
            ### Step 3: Decimal: Convert the binary operated values to a digit.
            where_img01_vector = np.where(img01_vector)[0]
            if len(where_img01_vector) >= 1:
                num = np.sum(2**where_img01_vector)
            else:
                num = 0
            imgLBP[ih+1,iw+1] = num
    return(imgLBP)

def create_training_data_from_circle():
    imagePaths = getListOfFiles("../data/raw") ## Folder structure: datasets --> sub-folders with labels name
    #print(imagePaths)

    data = []
    lables = []
    c = 0 ## to see the progress
    for image in imagePaths:

        lable = os.path.split(os.path.split(image)[0])[1]
        lables.append(lable)

#         img = cv2.imread(image)
    #     print('image is ', image)

#         img = getLBPimage(img)
#         circles, num_cirlces, img = detect_outside_circle(str(image), draw_circle=False)

#         merged_image = extract_infor_circle(img, circles[0][0][0], circles[0][0][1], circles[0][0][2])
#         img    = getLBPimage(merged_image)
        img = circle_to_lbp(image)
        img = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)        
#         vecimgLBP = imgLBP.flatten()
        data.append(img[0])
        c=c+1
        print(c)
    return data, lables

In [7]:
data, lables = create_training_data_from_circle()

The number of circle is  1
1
The number of circle is  1
2
The number of circle is  1
3
The number of circle is  1
4
The number of circle is  1
5
The number of circle is  1
6
The number of circle is  1
7
The number of circle is  1
8
The number of circle is  1
9
The number of circle is  1
10
The number of circle is  1
11
The number of circle is  1
12
The number of circle is  1
13
The number of circle is  1
14
The number of circle is  1
15
The number of circle is  1
16
The number of circle is  1
17
The number of circle is  1
18
The number of circle is  1
19
The number of circle is  1
20
The number of circle is  1
21
The number of circle is  1
22
The number of circle is  1
23
The number of circle is  1
24
The number of circle is  1
25
The number of circle is  1
26
The number of circle is  1
27
The number of circle is  1
28
The number of circle is  1
29


In [64]:
data = np.array(data)
lables = np.array(lables)

le = LabelEncoder()
lables = le.fit_transform(lables)

myset = set(lables)
print(myset)

dataset_size = data.shape[0]
data = data.reshape(dataset_size,-1)

print(data.shape)
print(lables.shape)
print(dataset_size)

(trainX, testX, trainY, testY ) = train_test_split(data, lables, test_size= 0.25, random_state=42)

model = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
model.fit(trainX, trainY)

print(classification_report(testY, model.predict(testX), target_names=le.classes_))

{0, 1}
(29, 200)
(29,)
29
              precision    recall  f1-score   support

          dn       0.67      0.80      0.73         5
          pc       0.50      0.33      0.40         3

    accuracy                           0.62         8
   macro avg       0.58      0.57      0.56         8
weighted avg       0.60      0.62      0.60         8



In [65]:
score = model.score(testX, testY)
score

0.625

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, trainX, trainY, cv=10)
clf.fit(trainX, trainY)
score = clf.score(testX, testY)
score

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(trainX, trainY)
clf.score(testX, testY)